# Partisan web scraping

In [2]:
from bs4 import BeautifulSoup
import urllib.request
import re
import json
import pandas as pd
import time
from datetime import date
import os
import requests
import tqdm
import ipywidgets
import time
from multiprocessing import Pool, set_start_method
# In download_partisans_html.py there is the function download_partisans_html which in this script is called
# by paralleled processes
from download_partisans_html import download_partisans_html

In [3]:
set_start_method('spawn')

In [4]:
today = str(date.today())
today

'2021-06-15'

In [23]:
#Основной парсер для партизан
partisans = []
links = os.listdir("/Users/annalevina/Desktop/links")
for link in links:
  if link == '.DS_Store':
    continue
  with open(os.path.join('/Users/annalevina/Desktop/links', link)) as f:
    soup = BeautifulSoup(f.read())
  credentials = {}
  name = soup.find("h1", attrs={'class': "info-book__title"}).text.strip()
  name = name.split(sep = " ", maxsplit=3)
  bio = soup.find("ul", attrs={'class': "info-book__descr"}).text
  credentials["original_link"] = "https://partizany.by/partisans/" + link + "/"
  credentials["retrieved_at"] = today         
  try:
    credentials["surname"] = name[0]
  except IndexError:
    credentials["surname"] = ""
  try:
    credentials["firstname"] = name[1]
  except IndexError:
    credentials["firstname"] = ""
  try:
    credentials["patronym"] = name[2]
  except IndexError:
    credentials["patronym"] = ""
  try:
    credentials["birthdate"] = re.search(r'\d{4}', bio).group().strip()
  except AttributeError:
    credentials["birthdate"] = ""
  try:
    credentials["birthplace"] = re.search(r'(?<=Место\sрождения: \s).*', bio).group().strip()
  except AttributeError:
    credentials["birthplace"] = ""
  try:
    credentials["nationality"] = re.search(r'(?<=Национальность:\s).*', bio).group().strip()
  except AttributeError:
    credentials["nationality"] = ""
  try:
    credentials["award"] = []
    award = soup.find("div", attrs={'class': "info-book__rewards-list"})
    for rewards_item in award.find_all("div", attrs={'class': "info-book__rewards-item"}):
      credentials["award"].append(rewards_item.find("img")['alt'])
  except AttributeError:
    credentials["award"] = ""
  try:
    credentials["award_nomination"] = []
    presented = soup.find("div", attrs={'class': "info-book__rewards-inline"})
    for item in presented.find_all("span", attrs={'class': "info-book__rewards-inline-text"}):
      credentials["award_nomination"].append(item.text)
  except AttributeError:
    credentials["award_nomination"] = "" 
  try:
    credentials["time_period"] = []
    credentials["partisan_brigade"] = []
    credentials["partisan_detachment"] = []
    credentials["position"] = []
    formation__list = soup.find("div", attrs={'class': "formation__list"})
    for formation_item in formation__list.find_all("div", attrs={'class': "formation__item"}):
      credentials["time_period"].append(formation_item.find("div", attrs={'class': "formation__item-date"}).text)
      for item_descr in formation_item.find_all("ul", attrs={'class': "formation__item-descr"}):
        help = item_descr.find_all("li")
        credentials["partisan_brigade"].append(help[0].find('a').text)
        credentials["partisan_detachment"].append(help[1].find('a').text)
        credentials["position"].append(re.sub("^\nДолжность: ", "", help[2].text).strip())
  except:  
    credentials["time_period"] = ""
    credentials["partisan_brigade"] = ""
    credentials["partisan_detachment"] = ""
    credentials["position"] = ""
  partisans.append(credentials)

AttributeError: 'NoneType' object has no attribute 'text'

In [11]:
partisans

[{'original_link': 'https://partizany.by/partisans/110588.html/',
  'retrieved_at': '2021-06-15',
  'surname': 'Альшевская',
  'firstname': 'Анастасия',
  'patronym': 'Васильевна',
  'birthdate': '1915',
  'birthplace': 'д.Стрикели, Полоцкий р-н, Витебская область, БССР',
  'nationality': 'Белорус',
  'award': ['Орден Отечественной войны II-й степени'],
  'award_nomination': ['Орден Красной Звезды   '],
  'time_period': ['С 21.09.1942 по  04.12.1943'],
  'partisan_brigade': ['3-я Белорусская бригада (Марченко, А.Я., Жиганов, В.В.)'],
  'partisan_detachment': ['6-й отряд (Шервашидзе, Г.К., Тимонин, А.М.)'],
  'position': ['Разведчик']},
 {'original_link': 'https://partizany.by/partisans/51428.html/',
  'retrieved_at': '2021-06-15',
  'surname': 'Ермолов',
  'firstname': 'Иван',
  'patronym': 'Ефимович',
  'birthdate': '1909',
  'birthplace': 'д.Староложниково, Городищенский р-н, Новосибирская область, РСФСР',
  'nationality': 'Русский',
  'award': '',
  'award_nomination': ['Медаль «Пар

In [ ]:
bio_parse = pd.DataFrame(partisans)
bio_parse

In [ ]:
with open('partisans.json', 'w', encoding='utf-8') as f:
    json.dump(partisans, f, ensure_ascii=False, indent=4)

In [3]:
if __name__ ==  '__main__': 
    # Initiate 30 processes running simultaneously
    p = Pool(30)
    numbers_considered = range(64936, 170000)
    number_processed = []
    for number in tqdm.tqdm(p.imap(download_partisans_html, numbers_considered), total=len(numbers_considered)):
        number_processed.append(number)

100%|██████████| 105064/105064 [4:29:20<00:00,  6.50it/s]    


In [ ]:
import requests
import time
from bs4 import BeautifulSoup

def download_partisans_html(number):
    page = "http://partizany.by/partisans/" + str(number)
    while True:
        try:
            page = requests.get(page, verify=False)
            break
        except:
            time.sleep(60)
    soup = BeautifulSoup(page.text)
    # If class = page_404  exists, partisan's page does not exist
    if soup.find("div", {"class": "page-404__text"}) is None:
        with open("links/" + str(number) + ".html", "w", encoding='utf-8') as f:
            f.write(page.text)
    return number